In [2]:
import pickle as pkl
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

import rasterio

import spacy
  
nlp = spacy.load('en_core_web_lg')
from sklearn.metrics.pairwise import cosine_similarity

import itertools

# Get Data

## Changes from year to year

In [3]:
changes = pd.read_csv("../../data/companyData/compustatChanges_2010s.csv").drop(columns = ['Unnamed: 0'])

changes.head()

,year,qtr,gvkey,companyName,tic,naics,curcdq,incomeChange,revenueChange,revenueChangeAbsolute,costChange,inventoryChange
0,2010,1.0,1004,AAR CORP,AIR,423860.0,USD,0.213983,0.200565,81.107,0.178258,0.055631
1,2010,2.0,1004,AAR CORP,AIR,423860.0,USD,0.045617,0.059577,27.099,0.017636,0.086776
2,2010,3.0,1004,AAR CORP,AIR,423860.0,USD,0.153198,0.166276,76.160,0.129456,0.169270
3,2010,4.0,1004,AAR CORP,AIR,423860.0,USD,-0.398739,0.174283,85.020,0.136013,0.182304
4,2011,1.0,1004,AAR CORP,AIR,423860.0,USD,0.096386,0.133883,65.000,0.101523,0.104150


In [4]:
industries = changes[['gvkey','naics']].drop_duplicates()
industries['naics'] = industries.naics.astype('str').str.slice(0,2)

## SC Linking Table for 2010s

In [ ]:
c_links = pd.read_csv("../../data/companyData/compustatSCLinked.csv")

c_links['year'] = c_links.srcdate.astype('str').str.slice(0,4).astype('int64')

c_links = c_links[c_links.year > 2009][['year','gvkey','cgvkey','salecs']].\
    rename(columns = {'cgvkey': 'customer_gvkey','gvkey': 'supplier_gvkey'})


print(c_links.shape)

c_links.head()

In [ ]:
industries.columns = ['customer_gvkey','customer_naics']

c_links = c_links.merge(industries)

industries.columns = ['supplier_gvkey','supplier_naics']

c_links = c_links.merge(industries)
c_links.head()

In [ ]:
c_links.shape

Now see if it's common to have one in and one out of the industries of interest.

In [ ]:
ofInterest = ['11','21','22','23','31','32','33','42','44','45','48','49']

c_linksCut = c_links[~(c_links.customer_naics.isin(ofInterest) & c_links.supplier_naics.isin(ofInterest))]

In [ ]:
c_linksCut['relat'] = c_linksCut.customer_naics + "_" + c_linksCut.supplier_naics
c_linksCut.relat.value_counts()[0:10]

In [ ]:
c_linksCut.supplier_naics.value_counts()[0:10]

## Compustat and ABI Linking

In [ ]:
gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])


base_columns = gvKey_abiLinkingTable.columns 
customer_columns = "customer_" + base_columns
supplier_columns = "supplier_" + base_columns


gvKey_abiLinkingTable.head()

In [ ]:
hasMatch = gvKey_abiLinkingTable.gvkey.unique()

In [ ]:
sum(c_links.supplier_gvkey.isin(hasMatch) | c_links.customer_gvkey.isin(hasMatch))

# Merge

In [ ]:

#########################
# merge in customer information
gvKey_abiLinkingTable.columns = customer_columns

print(c_links.shape)
c_linksMerge1 = c_links.merge(gvKey_abiLinkingTable, on ='customer_gvkey')
print(c_links.shape,c_linksMerge1.shape)



#########################
# and merge in supplier 
gvKey_abiLinkingTable.columns = supplier_columns

print(c_links.shape)
c_linksMerge2 = c_linksMerge1.merge(gvKey_abiLinkingTable, on ='supplier_gvkey')
print(c_links.shape,c_linksMerge2.shape)

In [ ]:
c_linksMerge2.to_csv("../../data/companyData/clinks_IG_selected.csv")

In [ ]:
gvKey_abiLinkingTable

This is probably because: (1) companies are not in North America, or (2) companies are not in the physical goods industries we're interested in. We can verify this though: look at c_links where both the customer and supplier are in the dataset of interest.

In [ ]:
chq     = pd.read_csv("../../data/chq.csv",dtype={'cstatZipcode': 'object'}).drop(columns = {'Unnamed: 0'})

c_linkTest = c_links[c_links.customer_gvkey.isin(chq.gvkey.unique()) & \
                     c_links.supplier_gvkey.isin(chq.gvkey.unique())]

print("Percent of firms with a match: ", c_linksMerge2.shape[0]/c_linkTest.shape[0])

It's entirely possible that we have too small of a sample from the 2010s alone. Let's just try it though and see how it goes.

First, make a sample with the companies on three years of either side of when it reports another customer.

In [ ]:
scTableCustomers = c_linksMerge2.copy()[['year','customer_gvkey','customer_abi']].drop_duplicates()
scTableSuppliers = c_linksMerge2.copy()[['year','supplier_gvkey','supplier_abi']].drop_duplicates()

In [ ]:
scTableCustomers.head()

In [ ]:
def makeThreeEitherSide(df): 
    yrPlus1 = df.copy(); yrPlus1['year'] += 1
    yrPlus2 = df.copy(); yrPlus2['year'] += 1
    yrPlus3 = df.copy(); yrPlus3['year'] += 1
    
    yrMinus1 = df.copy(); yrMinus1['year'] -= 1
    yrMinus2 = df.copy(); yrMinus2['year'] -= 1
    yrMinus3 = df.copy(); yrMinus3['year'] -= 1
    
    all = pd.concat([yrPlus1,yrPlus2,yrPlus3,yrMinus1,yrMinus2,yrMinus3])
    
    return(all)

In [ ]:
allCustomerData = makeThreeEitherSide(scTableCustomers)
allCustomerData.columns = ['year','gvkey','abi']


allSupplierData = makeThreeEitherSide(scTableSuppliers)
allSupplierData.columns = ['year','gvkey','abi']

In [ ]:
allSupplierData.year

In [ ]:
allAbi = allCustomerData.abi.append(allSupplierData.abi).drop_duplicates()

In [ ]:
hqsOnly = pd.read_csv("../../data/ig2010s_uniqueHQs.csv").drop(columns = {'Unnamed: 0'})

In [ ]:
hqsOnly.head()

In [ ]:
hq = pd.read_csv("../../data/ig2010s_uniqueHQs_multLocations.csv").\
    drop(columns = {'Unnamed: 0'}).\
    rename(columns = {'archive_version_year': 'year'})

hq['year'] = hq.year.astype('int64')

hqRelevant = hq[hq.abi.isin(allAbi)]

In [ ]:
hqRelevant[hqRelevant.abi == 71340]

In [ ]:
allSupplierData = allSupplierData.merge(hqRelevant).drop_duplicates()
allCustomerData = allCustomerData.merge(hqRelevant).drop_duplicates()

In [ ]:
allSupplierData.head()

In [ ]:
allCustomerData.to_csv("../../data/companyData/allCustomerData.csv")
allSupplierData.to_csv("../../data/companyData/allSupplierData.csv")

In [ ]:
allSupplierData.shape

In [ ]:
hqsOnly = pd.read_csv("../../data/ig2010s_uniqueHQs.csv")